In [1]:
import os
import shutil
import random
import numpy as np
import cv2
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
from ultralytics import YOLO
import logging
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split

In [2]:
# --- Setup Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
logger.info("Random seeds set for reproducibility.")

2025-04-11 11:14:43,816 - INFO - Random seeds set for reproducibility.


In [3]:
# --- Paths ---
base_dir = Path('../datasets').resolve()
cvat_dir = base_dir / 'cvat_upload'
cvat_images_dir = cvat_dir / 'images'
cvat_labels_dir = cvat_dir / 'labels'

yolo_dir = Path('../yolo_cvat_v4').resolve()
train_img_dir = yolo_dir / 'images' / 'train'
val_img_dir = yolo_dir / 'images' / 'val'
test_img_dir = yolo_dir / 'images' / 'test'  
train_lbl_dir = yolo_dir / 'labels' / 'train'
val_lbl_dir = yolo_dir / 'labels' / 'val'
test_lbl_dir = yolo_dir / 'labels' / 'test' 

for d in [train_img_dir, val_img_dir, test_img_dir, train_lbl_dir, val_lbl_dir, test_lbl_dir]:
    d.mkdir(parents=True, exist_ok=True)
logger.info("Directory structure created.")

2025-04-11 11:14:43,825 - INFO - Directory structure created.


In [4]:
# --- Collect and Split Data ---
images = list(cvat_images_dir.glob('*.jpg'))
labels = list(cvat_labels_dir.glob('*.txt'))

# Identify images without labels
images_without_labels = [img for img in images if not (cvat_labels_dir / f"{img.stem}.txt").exists()]
logger.info(f"Found {len(images_without_labels)} images without labels.")

# Create empty label files for missing annotations
for img in images_without_labels:
    empty_lbl = cvat_labels_dir / f"{img.stem}.txt"
    open(empty_lbl, 'w').close()
    logger.info(f"Created empty label file: {empty_lbl}")

# Split data into train, val, and test
train_imgs, temp_imgs = train_test_split(images, test_size=0.3, random_state=42)  # 70% train, 30% temp
val_imgs, test_imgs = train_test_split(temp_imgs, test_size=0.5, random_state=42)  # Split temp into 15% val, 15% test
logger.info(f"Collected {len(images)} images. Train: {len(train_imgs)}, Val: {len(val_imgs)}, Test: {len(test_imgs)}")


2025-04-11 11:14:43,868 - INFO - Found 0 images without labels.
2025-04-11 11:14:43,870 - INFO - Collected 2681 images. Train: 1876, Val: 402, Test: 403


In [5]:
# Copy images and labels to train/val/test directories
for split_imgs, img_dst, lbl_dst in [(train_imgs, train_img_dir, train_lbl_dir), 
                                     (val_imgs, val_img_dir, val_lbl_dir),
                                     (test_imgs, test_img_dir, test_lbl_dir)]:
    for img in tqdm(split_imgs, desc=f"Copying to {img_dst.parent.name}"):
        lbl = cvat_labels_dir / f"{img.stem}.txt"
        shutil.copy(img, img_dst / img.name)
        if lbl.exists():
            shutil.copy(lbl, lbl_dst / lbl.name)
        else:
            open(lbl_dst / f"{img.stem}.txt", 'w').close()
logger.info("Data split and copied to train/val/test directories.")


Copying to images: 100%|██████████| 403/403 [00:00<00:00, 740.73it/s]
2025-04-11 11:14:47,827 - INFO - Data split and copied to train/val/test directories.


In [6]:
# --- Define Classes ---
selected_classes = ['hole', 'pole', 'stairs', 'bottle', 'rock']
logger.info(f"Defined classes: {selected_classes}")

2025-04-11 11:14:47,834 - INFO - Defined classes: ['hole', 'pole', 'stairs', 'bottle', 'rock']


In [7]:
# --- Annotation Validation Functions ---
def collect_bbox_stats(label_dir, class_names):
    """Collect bounding box width and height statistics per class."""
    class_bboxes = defaultdict(list)
    for lbl_file in label_dir.glob('*.txt'):
        with open(lbl_file, 'r') as f:
            lines = f.readlines()
        for line in lines:
            if line.strip():
                parts = line.split()
                try:
                    cls_id = int(parts[0])
                    if cls_id >= len(class_names) or cls_id < 0:
                        logger.warning(f"Skipping invalid class ID {cls_id} in {lbl_file}")
                        continue
                    width = float(parts[3])
                    height = float(parts[4])
                    class_bboxes[cls_id].append((width, height))
                except ValueError:
                    logger.error(f"Invalid values in {lbl_file}: {line}")
    return class_bboxes

def compute_bbox_stats(class_bboxes):
    """Compute mean and std of bbox dimensions per class."""
    stats = {}
    for cls_id, bboxes in class_bboxes.items():
        if bboxes:
            widths = [w for w, h in bboxes]
            heights = [h for w, h in bboxes]
            stats[cls_id] = {
                'mean_w': np.mean(widths), 'std_w': np.std(widths),
                'mean_h': np.mean(heights), 'std_h': np.std(heights)
            }
        else:
            stats[cls_id] = {'mean_w': 0, 'std_w': 0, 'mean_h': 0, 'std_h': 0}
    return stats

def compute_iou(box1, box2):
    """Compute Intersection over Union between two bounding boxes."""
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    x1_min, x1_max = x1 - w1 / 2, x1 + w1 / 2
    y1_min, y1_max = y1 - h1 / 2, y1 + h1 / 2
    x2_min, x2_max = x2 - w2 / 2, x2 + w2 / 2
    y2_min, y2_max = y2 - h2 / 2, y2 + h2 / 2
    inter_x_min = max(x1_min, x2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_min = max(y1_min, y2_min)
    inter_y_max = min(y1_max, y2_max)
    inter_area = max(0, inter_x_max - inter_x_min) * max(0, inter_y_max - inter_y_min)
    area1 = w1 * h1
    area2 = w2 * h2
    union_area = area1 + area2 - inter_area
    return inter_area / union_area if union_area > 0 else 0

def check_annotations(label_dir, class_names, stats, iou_threshold=0.9, size_threshold=2):
    """Validate annotations for errors: invalid IDs, duplicates, size anomalies."""
    mislabel_images = []
    for lbl_file in label_dir.glob('*.txt'):
        with open(lbl_file, 'r') as f:
            lines = f.readlines()
        boxes = []
        for line in lines:
            if line.strip():
                parts = line.split()
                if len(parts) != 5:
                    logger.error(f"Invalid format in {lbl_file}: {line}")
                    continue
                try:
                    cls_id = int(parts[0])
                    x, y, w, h = map(float, parts[1:])
                except ValueError:
                    logger.error(f"Invalid values in {lbl_file}: {line}")
                    continue
                if cls_id >= len(class_names) or cls_id < 0:
                    logger.error(f"Invalid class ID {cls_id} in {lbl_file}")
                    continue
                if not (0 <= x <= 1 and 0 <= y <= 1 and 0 < w <= 1 and 0 < h <= 1):
                    logger.error(f"Out-of-range value in {lbl_file}: {line}")
                    continue
                boxes.append((cls_id, x, y, w, h))
                if cls_id in stats and stats[cls_id]['std_w'] > 0 and stats[cls_id]['std_h'] > 0:
                    mean_w, std_w = stats[cls_id]['mean_w'], stats[cls_id]['std_w']
                    mean_h, std_h = stats[cls_id]['mean_h'], stats[cls_id]['std_h']
                    if abs(w - mean_w) > size_threshold * std_w or abs(h - mean_h) > size_threshold * std_h:
                        logger.warning(f"Potential mislabel in {lbl_file}: {line.strip()} (class {class_names[cls_id]})")
                        mislabel_images.append(lbl_file.stem)
        for i in range(len(boxes)):
            for j in range(i + 1, len(boxes)):
                box1 = boxes[i][1:]
                box2 = boxes[j][1:]
                iou = compute_iou(box1, box2)
                if iou > iou_threshold:
                    logger.warning(f"High IoU ({iou:.2f}) in {lbl_file}: {boxes[i]} and {boxes[j]}")
                    mislabel_images.append(lbl_file.stem)
    logger.info(f"Annotation check completed for {label_dir}")
    return list(set(mislabel_images))

# --- Visualize Annotations ---
def visualize_annotations(image_path, label_path, class_names):
    """Visualize bounding boxes on an image."""
    img = cv2.imread(str(image_path))
    if img is None:
        logger.error(f"Failed to load image: {image_path}")
        return None
    h, w = img.shape[:2]
    with open(label_path, 'r') as f:
        for line in f:
            if line.strip():
                cls_id, x, y, width, height = map(float, line.split())
                x1 = int((x - width / 2) * w)
                y1 = int((y - height / 2) * h)
                x2 = int((x + width / 2) * w)
                y2 = int((y + height / 2) * h)
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                label = class_names[int(cls_id)]
                cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


def plot_samples_per_class(image_dir, label_dir, class_names, num_samples=3, output_dir='sample_visualizations'):
    """Plot sample images with annotations for each class and save to output_dir."""
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    class_images = {i: [] for i in range(len(class_names))}
    for lbl_file in label_dir.glob('*.txt'):
        with open(lbl_file, 'r') as f:
            lines = f.readlines()
        classes_in_image = set()
        for line in lines:
            if line.strip():
                cls_id = int(line.split()[0])
                classes_in_image.add(cls_id)
        for c_id in classes_in_image:
            class_images[c_id].append(lbl_file.stem)
    
    for cls_id, stems in class_images.items():
        if not stems:
            logger.warning(f"No images found for class {class_names[cls_id]} in {label_dir}")
            continue
        selected_stems = random.sample(stems, min(num_samples, len(stems)))
        for s in selected_stems:
            img_path = image_dir / f"{s}.jpg"
            lbl_path = label_dir / f"{s}.txt"
            annotated = visualize_annotations(img_path, lbl_path, class_names)
            if annotated is not None:
                plt.figure(figsize=(8, 6))
                plt.imshow(annotated)
                plt.title(f"{class_names[cls_id]} - {s}")
                plt.axis('off')
                safe_class_name = class_names[cls_id].replace("/", "_")
                plt.savefig(output_path / f"{safe_class_name}_{s}.png")
                plt.close()
    logger.info(f"Saved annotation visualizations for {label_dir.parent.name} in '{output_dir}'.")

def visualize_mislabels(image_dir, label_dir, class_names, mislabel_stems, output_dir='mislabels'):
    """Visualize images with potential mislabels and save them to a folder."""
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    for stem in mislabel_stems:
        img_path = image_dir / f"{stem}.jpg"
        lbl_path = label_dir / f"{stem}.txt"
        if img_path.exists() and lbl_path.exists():
            annotated = visualize_annotations(img_path, lbl_path, class_names)
            if annotated is not None:
                plt.figure(figsize=(8, 6))
                plt.imshow(annotated)
                plt.title(f"Potential Mislabel - {stem}")
                plt.axis('off')
                plt.savefig(output_path / f"{stem}.png")
                plt.close()
    logger.info(f"Saved visualizations of potential mislabels in '{output_dir}'.")

# Collect stats and check annotations
train_class_bboxes = collect_bbox_stats(train_lbl_dir, selected_classes)
train_stats = compute_bbox_stats(train_class_bboxes)
logger.info("Collected bounding box statistics from training set.")
train_mislabels = check_annotations(train_lbl_dir, selected_classes, train_stats)
val_mislabels = check_annotations(val_lbl_dir, selected_classes, train_stats)
test_mislabels = check_annotations(test_lbl_dir, selected_classes, train_stats)  # Check test set too
all_mislabels = list(set(train_mislabels + val_mislabels + test_mislabels))

2025-04-11 11:14:47,931 - INFO - Collected bounding box statistics from training set.
2025-04-11 11:14:47,933 - WARNING - Potential mislabel in /home/r0jin/projects/EnigmaAI/yolo_cvat_v4/labels/train/frame_IMG_4941_00002.txt: 1 0.107852 0.255609 0.208672 0.508406 (class pole)
2025-04-11 11:14:47,934 - WARNING - Potential mislabel in /home/r0jin/projects/EnigmaAI/yolo_cvat_v4/labels/train/frame_IMG_4318_00030.txt: 1 0.778164 0.206961 0.263359 0.413922 (class pole)
2025-04-11 11:14:47,934 - WARNING - Potential mislabel in /home/r0jin/projects/EnigmaAI/yolo_cvat_v4/labels/train/frame_IMG_4318_00030.txt: 1 0.237406 0.647367 0.474000 0.705266 (class pole)
2025-04-11 11:14:47,935 - WARNING - Potential mislabel in /home/r0jin/projects/EnigmaAI/yolo_cvat_v4/labels/train/frame_IMG_4912_00009.txt: 1 0.961836 0.280055 0.076328 0.392297 (class pole)
2025-04-11 11:14:47,937 - WARNING - Potential mislabel in /home/r0jin/projects/EnigmaAI/yolo_cvat_v4/labels/train/frame_IMG_4982_00004.txt: 0 0.583570

In [8]:
# Visualize all mislabels
def get_image_label_dirs(stem):
    if (train_img_dir / f"{stem}.jpg").exists():
        return train_img_dir, train_lbl_dir
    elif (val_img_dir / f"{stem}.jpg").exists():
        return val_img_dir, val_lbl_dir
    elif (test_img_dir / f"{stem}.jpg").exists():  # Include test set
        return test_img_dir, test_lbl_dir
    else:
        return None, None

for stem in all_mislabels:
    img_dir, lbl_dir = get_image_label_dirs(stem)
    if img_dir and lbl_dir:
        visualize_mislabels(img_dir, lbl_dir, selected_classes, [stem])

2025-04-11 11:14:48,257 - INFO - Saved visualizations of potential mislabels in 'mislabels'.
2025-04-11 11:14:48,394 - INFO - Saved visualizations of potential mislabels in 'mislabels'.
2025-04-11 11:14:48,533 - INFO - Saved visualizations of potential mislabels in 'mislabels'.
2025-04-11 11:14:48,681 - INFO - Saved visualizations of potential mislabels in 'mislabels'.
2025-04-11 11:14:48,822 - INFO - Saved visualizations of potential mislabels in 'mislabels'.
2025-04-11 11:14:48,953 - INFO - Saved visualizations of potential mislabels in 'mislabels'.
2025-04-11 11:14:49,092 - INFO - Saved visualizations of potential mislabels in 'mislabels'.
2025-04-11 11:14:49,243 - INFO - Saved visualizations of potential mislabels in 'mislabels'.
2025-04-11 11:14:49,383 - INFO - Saved visualizations of potential mislabels in 'mislabels'.
2025-04-11 11:14:49,528 - INFO - Saved visualizations of potential mislabels in 'mislabels'.
2025-04-11 11:14:49,658 - INFO - Saved visualizations of potential mis

In [9]:
# --- Class Distribution ---
def get_class_counts(label_dir):
    """Count instances per class in label files."""
    class_counts = Counter()
    for lbl_file in label_dir.glob('*.txt'):
        with open(lbl_file, 'r') as f:
            lines = f.readlines()
        for line in lines:
            if line.strip():
                cls_id = int(line.split()[0])
                class_counts[cls_id] += 1
    return class_counts

train_class_counts = get_class_counts(train_lbl_dir)
val_class_counts = get_class_counts(val_lbl_dir)
test_class_counts = get_class_counts(test_lbl_dir)  # Count test set classes
logger.info(f"Train class counts: {dict(train_class_counts)}")
logger.info(f"Val class counts: {dict(val_class_counts)}")
logger.info(f"Test class counts: {dict(test_class_counts)}")

plt.figure(figsize=(10, 5))
plt.bar(selected_classes, [train_class_counts.get(i, 0) for i in range(len(selected_classes))], 
        alpha=0.5, label='Train')
plt.bar(selected_classes, [val_class_counts.get(i, 0) for i in range(len(selected_classes))], 
        alpha=0.5, label='Val')
plt.bar(selected_classes, [test_class_counts.get(i, 0) for i in range(len(selected_classes))], 
        alpha=0.5, label='Test')  # Include test set in plot
plt.xlabel('Classes')
plt.ylabel('Number of Instances')
plt.title('Class Distribution')
plt.legend()
plt.savefig('class_distribution.png')
plt.close()
logger.info("Saved class distribution plot to 'class_distribution.png'.")


2025-04-11 11:15:04,358 - INFO - Train class counts: {4: 100, 1: 872, 3: 142, 2: 118, 0: 92}
2025-04-11 11:15:04,358 - INFO - Val class counts: {0: 12, 1: 195, 3: 21, 2: 27, 4: 12}
2025-04-11 11:15:04,359 - INFO - Test class counts: {0: 12, 1: 131, 3: 27, 4: 17, 2: 35}
2025-04-11 11:15:04,473 - INFO - Saved class distribution plot to 'class_distribution.png'.


In [10]:
# --- Visualize Test Set Predictions ---
def visualize_predictions(model, image_dir, label_dir, class_names, num_samples=5, output_dir='test_predictions'):
    """Visualize model predictions on test set images with ground truth and predictions."""
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    test_images = list(image_dir.glob('*.jpg'))
    selected_images = random.sample(test_images, min(num_samples, len(test_images)))
    
    for img_path in selected_images:
        # Load image
        img = cv2.imread(str(img_path))
        if img is None:
            logger.error(f"Failed to load image: {img_path}")
            continue
        
        # Get ground truth annotations
        lbl_path = label_dir / f"{img_path.stem}.txt"
        gt_boxes = []
        if lbl_path.exists():
            with open(lbl_path, 'r') as f:
                for line in f:
                    if line.strip():
                        cls_id, x, y, w, h = map(float, line.split())
                        gt_boxes.append((int(cls_id), x, y, w, h))
        
        # Run inference
        results = model(img_path)
        
        # Plot ground truth and predictions
        fig, ax = plt.subplots(1, 2, figsize=(16, 8))
        
        # Ground truth
        ax[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        ax[0].set_title("Ground Truth")
        h, w = img.shape[:2]
        for cls_id, x, y, width, height in gt_boxes:
            x1 = int((x - width / 2) * w)
            y1 = int((y - height / 2) * h)
            x2 = int((x + width / 2) * w)
            y2 = int((y + height / 2) * h)
            ax[0].add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1, edgecolor='green', facecolor='none', lw=2))
            ax[0].text(x1, y1 - 10, class_names[int(cls_id)], color='green', fontsize=12)
        
        # Predictions
        pred_img = results[0].plot()  # Plot predictions on the image
        ax[1].imshow(pred_img)
        ax[1].set_title("Predictions")
        
        # Save figure
        plt.savefig(output_path / f"{img_path.stem}_pred.png")
        plt.close()
    
    logger.info(f"Saved test set prediction visualizations in '{output_dir}'.")



In [11]:
# --- Create YOLO data.yaml ---
data_yaml_path = yolo_dir / 'data.yaml'
with open(data_yaml_path, 'w') as f:
    f.write(f"train: {train_img_dir.resolve()}\n")
    f.write(f"val: {val_img_dir.resolve()}\n")
    f.write(f"test: {test_img_dir.resolve()}\n")  
    f.write(f"nc: {len(selected_classes)}\n")
    f.write(f"names: {selected_classes}\n")
logger.info(f"Created data.yaml at {data_yaml_path}")

2025-04-11 11:15:04,492 - INFO - Created data.yaml at /home/r0jin/projects/EnigmaAI/yolo_cvat_v4/data.yaml


In [17]:
# --- Train YOLOv8 Model ---
model = YOLO('yolov8s.pt')
logger.info("Initialized YOLOv8s model from pretrained weights.")

results = model.train(
    data=str(data_yaml_path),
    epochs=100,
    imgsz=640,
    batch=8,
    lr0=0.0003,
    warmup_epochs=3,
    device='cuda' if torch.cuda.is_available() else 'cpu',
    name="cvat_v4_smoother",
    project="runs/train",
    patience=10,
    verbose=True,
    plots=True,
    augment=True,
    mosaic=0,
    mixup=0,
    degrees=5,
    translate=0.05,
    scale=0.1,
    shear=0.0,
    hsv_h=0.005,             # lower color jitter
    hsv_s=0.3,
    hsv_v=0.2
    )

2025-04-11 12:02:16,038 - INFO - Initialized YOLOv8s model from pretrained weights.


New https://pypi.org/project/ultralytics/8.3.107 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.105 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2070 with Max-Q Design, 7959MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/home/r0jin/projects/EnigmaAI/yolo_cvat_v4/data.yaml, epochs=100, time=None, patience=10, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=runs/train, name=cvat_v4_smoother, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, 

train: Scanning /home/r0jin/projects/EnigmaAI/yolo_cvat_v4/labels/train.cache... 1876 images, 1218 backgrounds, 0 corrupt: 100%|██████████| 1876/1876 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /home/r0jin/projects/EnigmaAI/yolo_cvat_v4/labels/val.cache... 402 images, 269 backgrounds, 0 corrupt: 100%|██████████| 402/402 [00:00<?, ?it/s]


...

     60/100      2.23G     0.9985     0.7935      1.122          2        640: 100%|██████████| 235/235 [00:36<00:00,  6.52it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:03<00:00,  6.97it/s]

                   all        402        267      0.689      0.677      0.703      0.458
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 50, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



60 epochs completed in 0.619 hours.
Optimizer stripped from runs/train/cvat_v4_smoother/weights/last.pt, 22.5MB
Optimizer stripped from runs/train/cvat_v4_smoother/weights/best.pt, 22.5MB

Validating runs/train/cvat_v4_smoother/weights/best.pt...
Ultralytics 8.3.105 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2070 with Max-Q Design, 7959MiB)
Model summary (fused): 72 layers, 11,127,519 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:07<00:00,  3.62it/s]


                   all        402        267      0.672      0.753      0.736      0.469
                  hole          9         12      0.628      0.583      0.512      0.247
                  pole         76        195      0.739      0.821       0.85      0.515
                stairs         25         27      0.559      0.741      0.713      0.436
                bottle         16         21      0.846      0.952      0.953      0.733
                  rock          9         12      0.587      0.667      0.651      0.415
Speed: 0.3ms preprocess, 14.6ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/train/cvat_v4_smoother


In [18]:
# --- Print Training Metrics ---
logger.info("Training Metrics:")
print(f"mAP50: {results.box.map50:.4f}")
print(f"mAP50-95: {results.box.map:.4f}")
print(f"Precision: {np.mean(results.box.p):.4f}")
print(f"Recall: {np.mean(results.box.r):.4f}")
for i, cls in enumerate(selected_classes):
    print(f"{cls} - mAP50: {results.box.maps[i]:.4f}")

2025-04-11 12:39:41,716 - INFO - Training Metrics:


mAP50: 0.7358
mAP50-95: 0.4691
Precision: 0.6717
Recall: 0.7527
hole - mAP50: 0.2466
pole - mAP50: 0.5153
stairs - mAP50: 0.4356
bottle - mAP50: 0.7329
rock - mAP50: 0.4152


In [19]:
# --- Additional Validation ---
val_results = model.val(data=str(data_yaml_path), split='val')
logger.info("Validation completed on validation set.")
print("\nValidation Metrics:")
print(val_results)

# --- Test Set Evaluation ---
test_results = model.val(data=str(data_yaml_path), split='test')
logger.info("Test set evaluation completed.")
print("\nTest Metrics:")
print(test_results)

# --- Export Model ---
model.export(format='onnx')
logger.info("Model exported to ONNX format.")

Ultralytics 8.3.105 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2070 with Max-Q Design, 7959MiB)
Model summary (fused): 72 layers, 11,127,519 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /home/r0jin/projects/EnigmaAI/yolo_cvat_v4/labels/val.cache... 402 images, 269 backgrounds, 0 corrupt: 100%|██████████| 402/402 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00:12<00:00,  4.08it/s]


                   all        402        267       0.67      0.753      0.735      0.469
                  hole          9         12      0.626      0.583      0.512      0.248
                  pole         76        195      0.734      0.821      0.845      0.515
                stairs         25         27      0.558      0.741      0.714      0.438
                bottle         16         21      0.846      0.952      0.953      0.732
                  rock          9         12      0.585      0.667      0.651      0.415
Speed: 0.6ms preprocess, 27.5ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/train/cvat_v4_smoother2


2025-04-11 12:39:57,613 - INFO - Validation completed on validation set.



Validation Metrics:
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7531bc6a8f10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046, 

val: Scanning /home/r0jin/projects/EnigmaAI/yolo_cvat_v4/labels/test.cache... 403 images, 281 backgrounds, 0 corrupt: 100%|██████████| 403/403 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00:11<00:00,  4.42it/s]


                   all        403        222      0.715      0.746      0.767      0.517
                  hole          7         12      0.502      0.583      0.545      0.316
                  pole         52        131      0.811      0.824      0.844       0.52
                stairs         31         35      0.747      0.714      0.748      0.507
                bottle         20         27      0.928      0.959      0.977       0.73
                  rock         14         17      0.588      0.647       0.72      0.514
Speed: 0.5ms preprocess, 25.4ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/train/cvat_v4_smoother3


2025-04-11 12:40:12,177 - INFO - Test set evaluation completed.



Test Metrics:
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7531bc6aa260>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0

2025-04-11 12:40:13,661 - INFO - Model exported to ONNX format.


In [20]:
# --- Visualize Test Set Predictions ---
def visualize_predictions(model, image_dir, label_dir, class_names, num_samples=5, output_dir='test_predictions'):
    """Visualize model predictions on test set images with ground truth and predictions."""
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    test_images = list(image_dir.glob('*.jpg'))
    selected_images = random.sample(test_images, min(num_samples, len(test_images)))
    
    for img_path in selected_images:
        # Load image
        img = cv2.imread(str(img_path))
        if img is None:
            logger.error(f"Failed to load image: {img_path}")
            continue
        
        # Get ground truth annotations
        lbl_path = label_dir / f"{img_path.stem}.txt"
        gt_boxes = []
        if lbl_path.exists():
            with open(lbl_path, 'r') as f:
                for line in f:
                    if line.strip():
                        cls_id, x, y, w, h = map(float, line.split())
                        gt_boxes.append((int(cls_id), x, y, w, h))
        
        # Run inference
        results = model(img_path)
        
        # Plot ground truth and predictions
        fig, ax = plt.subplots(1, 2, figsize=(16, 8))
        
        # Ground truth
        ax[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        ax[0].set_title("Ground Truth")
        h, w = img.shape[:2]
        for cls_id, x, y, width, height in gt_boxes:
            x1 = int((x - width / 2) * w)
            y1 = int((y - height / 2) * h)
            x2 = int((x + width / 2) * w)
            y2 = int((y + height / 2) * h)
            ax[0].add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1, edgecolor='green', facecolor='none', lw=2))
            ax[0].text(x1, y1 - 10, class_names[int(cls_id)], color='green', fontsize=12)
        
        # Predictions
        pred_img = results[0].plot()  # Plot predictions on the image
        ax[1].imshow(pred_img)
        ax[1].set_title("Predictions")
        
        # Save figure
        plt.savefig(output_path / f"{img_path.stem}_pred.png")
        plt.close()
    
    logger.info(f"Saved test set prediction visualizations in '{output_dir}'.")

# --- Call Visualizations ---
# Visualize sample annotations for all sets
plot_samples_per_class(train_img_dir, train_lbl_dir, selected_classes, num_samples=3, output_dir='train_visualizations')
plot_samples_per_class(val_img_dir, val_lbl_dir, selected_classes, num_samples=3, output_dir='val_visualizations')
plot_samples_per_class(test_img_dir, test_lbl_dir, selected_classes, num_samples=3, output_dir='test_visualizations')

# Visualize model predictions on test set
visualize_predictions(model, test_img_dir, test_lbl_dir, selected_classes, num_samples=5)

2025-04-11 12:40:16,189 - INFO - Saved annotation visualizations for labels in 'train_visualizations'.
2025-04-11 12:40:18,466 - INFO - Saved annotation visualizations for labels in 'val_visualizations'.
2025-04-11 12:40:20,725 - INFO - Saved annotation visualizations for labels in 'test_visualizations'.



image 1/1 /home/r0jin/projects/EnigmaAI/yolo_cvat_v4/images/test/frame_IMG_4925_00004.jpg: 640x640 (no detections), 33.6ms
Speed: 1.2ms preprocess, 33.6ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/r0jin/projects/EnigmaAI/yolo_cvat_v4/images/test/frame_IMG_4920_00008.jpg: 640x640 (no detections), 32.6ms
Speed: 1.3ms preprocess, 32.6ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/r0jin/projects/EnigmaAI/yolo_cvat_v4/images/test/frame_IMG_4331_00023.jpg: 640x640 (no detections), 32.6ms
Speed: 1.3ms preprocess, 32.6ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/r0jin/projects/EnigmaAI/yolo_cvat_v4/images/test/frame_IMG_4329_00021.jpg: 640x640 (no detections), 32.6ms
Speed: 1.3ms preprocess, 32.6ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/r0jin/projects/EnigmaAI/yolo_cvat_v4/images/test/frame_IMG_4941_00007.jpg: 640x640 2 poles, 32.5ms


2025-04-11 12:40:23,053 - INFO - Saved test set prediction visualizations in 'test_predictions'.
